## Demonstration of ogcapi API

The initial attempt to use the `owslib` library failed due to issus in the requests getting through to the F5.

Work from here down uses `requests` directly.

In [1]:
import folium
import requests

### Get some GeoJSON data and display it directly on a `folium` map.

In [2]:
# Get the data
params = {'f': 'json', 'limit': 10}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()

In [3]:
# Define an initial map centred on and zoomed to the UK
m = folium.Map(location=[55, -2], zoom_start=5)

In [4]:
# Use a style function to set the style dynamically per data point
# In this case use the magnitude to set the radius of the marker
def style_function(feature):
    radius = feature['properties']['ml'] * 5
    return {'radius': radius,
            'color': 'red',
            'weight': 2}

In [5]:
# Create a basic unstyled tooltip
tooltip = folium.features.GeoJsonTooltip(
    fields=['ml'],
    aliases=['magnitude'])

In [6]:
# Create a basic unstyled popup
popup = folium.features.GeoJsonPopup(
    fields=["year", "latitude", "longitude", "depth", "ml", "intensity"],
    aliases=["year", "latitude", "longitude", "depth", "magnitude", "intensity"])

In [7]:
# Populate and display the map
folium.GeoJson(
    data,
    tooltip=tooltip,
    popup=popup,
    style_function=style_function,
    marker=folium.CircleMarker()
).add_to(m)
m

### Now get the data into geopandas, transform it and then create a map

In [8]:
import geopandas as gpd
import pandas as pd
import branca.colormap as cm
from shapely.geometry import Polygon

In [9]:
# Get all the data in batches of 'limit'.
# This is a bit ugly, there may be a neater way.
earthquake_list = []
offset = 0
limit = 500
params = {'f': 'json', 'offset': offset, 'limit': limit}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()
while len(data['features']) == limit:
    # Get JSON into GeoPandas
    earthquake_list.append(gpd.GeoDataFrame.from_features(data['features']))
    # Get the next batch
    offset += limit
    params = {'f': 'json', 'offset': offset, 'limit': limit}
    response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
    data = response.json()

# Add the final batch if not empty
if len(data['features']) > 0:
    earthquake_list.append(gpd.GeoDataFrame.from_features(data['features']))

# Concatenate dataframes into one
earthquakes = pd.concat(
    earthquake_list,
    ignore_index=True,
)
# Set geometry
earthquakes = earthquakes.set_geometry('geometry')
earthquakes.tail()

,geometry,year,longitude,ml,intensity,date_entered,date_updated,latitude,datetime,depth,user_entered,user_updated
9923,POINT (-3.85200 53.13100),2022,-3.852,1.8,0.0,2022-10-13T06:00:01,None,53.131,2022-10-11T08:37:12,7.8,SYSTEM,None
9924,POINT (-3.85200 53.13100),2022,-3.852,1.8,0.0,2022-10-15T06:00:02,None,53.131,2022-10-11T08:37:12,7.8,SYSTEM,None
9925,POINT (-3.85200 53.13100),2022,-3.852,1.8,0.0,2022-10-17T06:00:00,None,53.131,2022-10-11T08:37:12,7.8,SYSTEM,None
9926,POINT (-3.85200 53.13100),2022,-3.852,1.8,0.0,2022-10-19T06:00:00,None,53.131,2022-10-11T08:37:12,7.8,SYSTEM,None
9927,POINT (-3.46100 56.15600),2022,-3.461,1.0,0.0,2022-10-21T06:00:00,None,56.156,2022-10-19T19:33:30,3.8,SYSTEM,None


In [10]:
earthquakes.dtypes

geometry        geometry
year              object
longitude        float64
ml               float64
intensity        float64
date_entered      object
date_updated      object
latitude         float64
datetime          object
depth            float64
user_entered      object
user_updated      object
dtype: object

In [11]:
# Rename, convert and select columns. Set the CRS.
earthquakes.rename(columns={'id': 'id', 'ml': 'magnitude'}, inplace=True)
# earthquakes['datetime'] = pd.to_datetime(earthquakes['datetime']) # leave as string for now
earthquakes['id'] = earthquakes['id'].astype(int)
earthquakes = earthquakes[['id', 'datetime', 'magnitude', 'intensity', 'depth', 'geometry']]
earthquakes.set_crs(epsg=4326, inplace=True)
earthquakes.head()

KeyError: 'id'

In [12]:
earthquakes.dtypes

geometry        geometry
year              object
longitude        float64
magnitude        float64
intensity        float64
date_entered      object
date_updated      object
latitude         float64
datetime          object
depth            float64
user_entered      object
user_updated      object
dtype: object

In [13]:
earthquakes.describe()

,longitude,magnitude,intensity,latitude,depth
count,9928.000000,9928.000000,9928.000000,9928.000000,9411.000000
mean,-2.384039,1.697774,0.247683,55.181075,7.441877
std,2.823507,0.639514,0.880600,3.010857,8.121938
min,-11.890000,1.000000,0.000000,49.001000,0.000000
25%,-4.368000,1.200000,0.000000,53.060000,1.900000
50%,-3.040000,1.500000,0.000000,55.098000,5.500000
75%,-1.200000,2.000000,0.000000,56.481250,10.400000
max,5.000000,5.400000,6.000000,63.000000,183.700000


In [14]:
# Get the earthquakes above magnitude 4
big_quakes = earthquakes[earthquakes['magnitude'] >= 4]
big_quakes

,geometry,year,longitude,magnitude,intensity,date_entered,date_updated,latitude,datetime,depth,user_entered,user_updated
44,POINT (-2.47000 54.50000),1970,-2.470,4.1,5.0,2018-08-29T17:29:10,None,54.500,1970-08-09T20:09:01,20.9,APBE,None
52,POINT (2.80000 59.40000),1971,2.800,4.7,0.0,2018-08-29T17:29:10,None,59.400,1971-03-23T20:05:18,NaN,APBE,None
113,POINT (-3.12000 51.64000),1974,-3.120,4.1,0.0,2018-08-29T17:29:11,None,51.640,1974-02-25T20:03:43,NaN,APBE,None
240,POINT (2.00000 62.00000),1976,2.000,4.2,0.0,2018-08-29T17:29:11,None,62.000,1976-08-18T20:45:52,NaN,APBE,None
289,POINT (3.00000 61.50000),1977,3.000,4.4,0.0,2018-08-29T17:29:11,None,61.500,1977-04-06T19:32:04,NaN,APBE,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9582,POINT (4.12900 61.80400),2022,4.129,4.2,0.0,2022-08-28T06:00:02,None,61.804,2022-08-26T01:47:11,19.1,SYSTEM,None
9646,POINT (-1.61100 52.28200),2000,-1.611,4.2,5.0,2022-08-30T06:00:00,None,52.282,2000-09-23T04:23:45,15.2,SYSTEM,None
9655,POINT (1.92700 59.94300),2000,1.927,4.5,0.0,2022-08-30T06:00:00,None,59.943,2000-12-08T05:54:01,9.8,SYSTEM,None
9688,POINT (3.25900 56.68200),2001,3.259,4.2,0.0,2022-08-30T06:00:00,None,56.682,2001-05-07T09:43:33,1.0,SYSTEM,None


In [15]:
# Use shapely to get the centre of the map
bounds = Polygon(list(big_quakes['geometry'])).bounds
bounds

(-10.904000000000044, 49.15299999999999, 5.000000000000001, 62.67000000000002)

In [16]:
mag_min = big_quakes['magnitude'].min()
mag_max = big_quakes['magnitude'].max()

In [17]:
# Create a colormap using the min and max magnitudes
colormap = cm.LinearColormap(colors=['yellow','red'], caption='Magnitude', vmin=mag_min, vmax=mag_max)

# Define a style function to apply that map
def map_style_function(feature):
    color = colormap(feature['properties']['magnitude'])
    return {'radius': 3,
            'color': color,
            'fill': True,
            'fill_opacity': 1}

# Define an initial map centred on the points
m2 = folium.Map(
    location=[(bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2],
    zoom_start=4)

# Create popup
popup = folium.features.GeoJsonPopup(fields=["datetime", "magnitude", "intensity", "depth"])

# Populate the map, converting the dataframe to JSON
folium.GeoJson(
    big_quakes.to_json(),
    popup=popup,
    style_function=map_style_function,
    marker=folium.CircleMarker()
).add_to(m2)

# Add colormap to map and display the map
m2.add_child(colormap)
m2